In [1]:
import torch
import numpy as np
import traceback
from module import *
def debug_generator_gatedcnn():
    # Test parameters
    batch_size = 4
    seq_length = 128
    input_dim = 24
    
    # Create random input tensor
    inputs = torch.randn(batch_size, seq_length, input_dim)
    print(f"\nInput tensor shape: {inputs.shape}")
    
    try:
        # Initialize model
        model = generator_gatedcnn(inputs=input_dim)
        model.eval()
        
        # Add hooks to print intermediate tensor shapes
        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.shape
            return hook
        
        # Register hooks for all major layers
        model.h1_conv.register_forward_hook(get_activation('h1_conv'))
        model.h1_conv_gates.register_forward_hook(get_activation('h1_conv_gates'))
        model.downsample1.register_forward_hook(get_activation('downsample1'))
        model.downsample2.register_forward_hook(get_activation('downsample2'))
        model.residual1.register_forward_hook(get_activation('residual1'))
        #model.transformer1.register_forward_hook(get_activation('transformer1'))
        model.residual2.register_forward_hook(get_activation('residual2'))
        #model.transformer2.register_forward_hook(get_activation('transformer2'))
        model.upsample1.register_forward_hook(get_activation('upsample1'))
        model.upsample2.register_forward_hook(get_activation('upsample2'))
        model.output_conv.register_forward_hook(get_activation('output_conv'))
        
        # Forward pass with detailed shape printing
        print("\nStarting forward pass...")
        print(f"Initial input shape: {inputs.shape}")
        
        x = model.input_transpose(inputs)
        print(f"After input transpose: {x.shape}")
        
        with torch.no_grad():
            try:
                outputs = model(inputs)
                print("\nSuccessful forward pass! Final output shape:", outputs.shape)
                
                print("\nIntermediate tensor shapes:")
                for name, shape in activation.items():
                    print(f"{name}: {shape}")
                
            except Exception as e:
                print("\n❌ Error during forward pass!")
                print("\nError message:", str(e))
                print("\nFull traceback:")
                traceback.print_exc()
                
                print("\nLast known tensor shapes before error:")
                for name, shape in activation.items():
                    print(f"{name}: {shape}")
                
    except Exception as e:
        print("\n❌ Error during model initialization!")
        print("Error message:", str(e))
        traceback.print_exc()

if __name__ == "__main__":
    debug_generator_gatedcnn()


Input tensor shape: torch.Size([4, 128, 24])

Starting forward pass...
Initial input shape: torch.Size([4, 128, 24])
After input transpose: torch.Size([4, 24, 128])

Successful forward pass! Final output shape: torch.Size([4, 8, 24])

Intermediate tensor shapes:
h1_conv: torch.Size([4, 128, 128])
h1_conv_gates: torch.Size([4, 128, 128])
downsample1: torch.Size([4, 256, 64])
downsample2: torch.Size([4, 512, 32])
residual1: torch.Size([4, 512, 32])
residual2: torch.Size([4, 512, 32])
upsample1: torch.Size([4, 2048, 16])
upsample2: torch.Size([4, 1024, 8])
output_conv: torch.Size([4, 24, 8])


c:\Users\naman\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "


In [2]:
import torch
import numpy as np
import pytest
import traceback
import sys
from typing import Any, Tuple
from module import *
class TestError(Exception):
    """Custom exception for test failures with detailed information"""
    pass

def run_test_with_trace(func: callable) -> Tuple[bool, str]:
    """
    Run a test function and return detailed trace information if it fails
    
    Args:
        func (callable): Test function to run
        
    Returns:
        Tuple[bool, str]: (success status, error message if any)
    """
    try:
        func()
        return True, ""
    except AssertionError as e:
        _, _, tb = sys.exc_info()
        tb_str = ''.join(traceback.format_tb(tb))
        return False, f"Assertion Error in {func.__name__}:\n{tb_str}\nError: {str(e)}"
    except Exception as e:
        _, _, tb = sys.exc_info()
        tb_str = ''.join(traceback.format_tb(tb))
        return False, f"Unexpected Error in {func.__name__}:\n{tb_str}\nError: {str(e)}"

def test_generator_gatedcnn():
    """Test suite for generator_gatedcnn with detailed error reporting"""
    
    # Test parameters
    batch_size = 4
    seq_length = 128
    input_dim = 24
    
    try:
        # Create random input tensor
        inputs = torch.randn(batch_size, seq_length, input_dim)
        
        # Initialize model
        try:
            model = generator_gatedcnn(inputs=input_dim)
        except Exception as e:
            raise TestError(f"Failed to initialize model:\n{traceback.format_exc()}")
        
        # Test functions with detailed error checking
        def test_model_initialization() -> None:
            """Test if model initializes without errors"""
            if not isinstance(model, generator_gatedcnn):
                raise TestError(f"Model is of type {type(model)}, expected generator_gatedcnn")
            print("✓ Model initialization test passed")
        
        def test_model_parameters() -> None:
            """Test if model has trainable parameters"""
            try:
                trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
                if trainable_params == 0:
                    raise TestError("Model has no trainable parameters")
                print(f"✓ Model has {trainable_params:,} trainable parameters")
            except Exception as e:
                raise TestError(f"Failed to check model parameters:\n{traceback.format_exc()}")
        
        def test_forward_pass() -> None:
            """Test if forward pass works and outputs correct shape"""
            try:
                model.eval()
                with torch.no_grad():
                    outputs = model(inputs)
                    
                    # Detailed shape checking
                    expected_output_shape = (4, 8, 24)
                    if outputs.shape != expected_output_shape:
                        raise TestError(
                            f"Output shape mismatch:\n"
                            f"Expected: {expected_output_shape}\n"
                            f"Got: {outputs.shape}\n"
                            f"Input shape was: {inputs.shape}"
                        )
                    
                    # Check for NaN/Inf with position information
                    nan_mask = torch.isnan(outputs)
                    inf_mask = torch.isinf(outputs)
                    
                    if nan_mask.any():
                        nan_positions = torch.nonzero(nan_mask)
                        raise TestError(f"NaN values found at positions:\n{nan_positions}")
                    
                    if inf_mask.any():
                        inf_positions = torch.nonzero(inf_mask)
                        raise TestError(f"Infinite values found at positions:\n{inf_positions}")
                    
                    print("✓ Forward pass test passed")
                    print(f"  Input shape: {inputs.shape}")
                    print(f"  Output shape: {outputs.shape}")
            except Exception as e:
                raise TestError(f"Forward pass failed:\n{traceback.format_exc()}")
        
        def test_device_compatibility() -> None:
            """Test if model works on both CPU and GPU"""
            try:
                if torch.cuda.is_available():
                    model_cuda = model.cuda()
                    inputs_cuda = inputs.cuda()
                    
                    with torch.no_grad():
                        outputs_cuda = model_cuda(inputs_cuda)
                    
                    if not outputs_cuda.is_cuda:
                        raise TestError("Output tensor is not on GPU")
                    print("✓ GPU compatibility test passed")
                else:
                    print("ℹ GPU not available, skipping GPU test")
            except Exception as e:
                raise TestError(f"GPU compatibility test failed:\n{traceback.format_exc()}")
        
        def test_gradient_flow() -> None:
            """Test if gradients flow through the model"""
            try:
                model.train()
                optimizer = torch.optim.Adam(model.parameters())
                
                outputs = model(inputs)
                loss = outputs.mean()
                
                optimizer.zero_grad()
                loss.backward()
                
                # Detailed gradient checking
                gradient_info = []
                has_gradients = False
                
                for name, param in model.named_parameters():
                    if param.requires_grad:
                        if param.grad is None:
                            gradient_info.append(f"No gradient for parameter: {name}")
                        else:
                            grad_sum = param.grad.abs().sum().item()
                            gradient_info.append(f"Gradient sum for {name}: {grad_sum}")
                            if grad_sum > 0:
                                has_gradients = True
                
                if not has_gradients:
                    raise TestError("No gradients are flowing through the model:\n" + 
                                  "\n".join(gradient_info))
                
                print("✓ Gradient flow test passed")
            except Exception as e:
                raise TestError(f"Gradient flow test failed:\n{traceback.format_exc()}")
        
        # Run all tests with detailed error reporting
        test_functions = [
            test_model_initialization,
            test_model_parameters,
            test_forward_pass,
            test_device_compatibility,
            test_gradient_flow
        ]
        
        print("\nRunning tests for generator_gatedcnn...")
        failed_tests = []
        
        for test_func in test_functions:
            success, error_msg = run_test_with_trace(test_func)
            if not success:
                failed_tests.append((test_func.__name__, error_msg))
        
        if failed_tests:
            print("\n❌ Some tests failed:")
            for test_name, error_msg in failed_tests:
                print(f"\nTest: {test_name}")
                print("Error:")
                print(error_msg)
        else:
            print("\nAll tests passed successfully! ✨")
        
    except Exception as e:
        print(f"\n❌ Critical error in test suite:")
        print(traceback.format_exc())

if __name__ == "__main__":
    test_generator_gatedcnn()


Running tests for generator_gatedcnn...
✓ Model initialization test passed
✓ Model has 27,275,032 trainable parameters
✓ Forward pass test passed
  Input shape: torch.Size([4, 128, 24])
  Output shape: torch.Size([4, 8, 24])
ℹ GPU not available, skipping GPU test


c:\Users\naman\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\instancenorm.py:88: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "


✓ Gradient flow test passed

All tests passed successfully! ✨
